In [13]:
import pandas as pd
import sqlite3

In [ ]:
# Creating and Connecting to SQLite database
conn = sqlite3.connect("fincra.db")

In [15]:
transactions = pd.read_csv(r"C:\Users\New-user\Downloads\transactions.csv")
merchants = pd.read_csv(r"C:\Users\New-user\Downloads\merchants.csv")
chargebacks = pd.read_csv(r"C:\Users\New-user\Downloads\chargebacks.csv")

In [17]:
transactions.head()

,transaction_id,merchant_id,transaction_date,transaction_amount,currency,payment_method,status,failure_reason,country,customer_id
0,TXN000001,M0079,2024-01-01 0:00:00,638.54,GBP,credit_card,successful,NaN,Nigeria,C00493
1,TXN000002,M0052,2024-01-01 1:00:00,404.26,USD,crypto,chargeback,NaN,Kenya,C00648
2,TXN000003,M0039,2024-01-01 2:00:00,27.81,EUR,credit_card,successful,NaN,Canada,C00711
3,TXN000004,M0097,2024-01-01 3:00:00,956.75,NGN,mobile_money,successful,NaN,UK,C00057
4,TXN000005,M0056,2024-01-01 4:00:00,459.85,GBP,crypto,successful,NaN,UK,C00735


In [19]:
merchants.head()

,merchant_id,merchant_name,merchant_category,country
0,M0001,Merchant_1,gaming,Kenya
1,M0002,Merchant_2,education,UK
2,M0003,Merchant_3,SaaS,Kenya
3,M0004,Merchant_4,travel,Nigeria
4,M0005,Merchant_5,travel,UK


In [21]:
chargebacks.head()

,chargeback_id,transaction_id,merchant_id,chargeback_date,chargeback_amount,reason
0,CB00001,TXN000002,M0052,2024-02-01,404.26,fraud
1,CB00002,TXN000009,M0094,2024-02-02,819.55,fraud
2,CB00003,TXN000012,M0007,2024-02-03,366.68,product not received
3,CB00004,TXN000032,M0043,2024-02-04,399.92,duplicate transaction
4,CB00005,TXN000066,M0020,2024-02-05,77.49,product not received


In [31]:
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])
transactions["date"] = transactions["transaction_date"].dt.date
transactions["time"] = transactions["transaction_date"].dt.strftime('%H:%M')  # Format time to HH:MM
transactions["month"] = transactions["transaction_date"].dt.strftime('%B')  # Get month as full name


In [33]:
# Classify time into Early Hours, Morning, Afternoon, Evening, and Night for analytics
def classify_time(hour):
    if 0 <= hour < 7:
        return "Early Hours"
    elif 7 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 16:
        return "Afternoon"
    elif 16 <= hour < 19:
        return "Evening"
    elif 19 <= hour < 24:
        return "Night"
    else:
        return "Invalid Time"

transactions["day_category"] = transactions["transaction_date"].dt.hour.apply(classify_time)

transactions["failure_reason"] = transactions["failure_reason"].fillna("NIL")

transactions.drop(columns=["transaction_date"], inplace=True)

# Capitalize merchant categories
merchants["merchant_category"] = merchants["merchant_category"].str.capitalize()

In [35]:
# Store cleaned data into SQLite database
transactions.to_sql("transactions", conn, if_exists="replace", index=False)
merchants.to_sql("merchants", conn, if_exists="replace", index=False)
chargebacks.to_sql("chargebacks", conn, if_exists="replace", index=False)

print("Data cleaning and storage complete!")


Data cleaning and storage complete!


In [37]:
transactions.head()

,transaction_id,merchant_id,transaction_amount,currency,payment_method,status,failure_reason,country,customer_id,date,time,month,day_category
0,TXN000001,M0079,638.54,GBP,credit_card,successful,NIL,Nigeria,C00493,2024-01-01,00:00,January,Early Hours
1,TXN000002,M0052,404.26,USD,crypto,chargeback,NIL,Kenya,C00648,2024-01-01,01:00,January,Early Hours
2,TXN000003,M0039,27.81,EUR,credit_card,successful,NIL,Canada,C00711,2024-01-01,02:00,January,Early Hours
3,TXN000004,M0097,956.75,NGN,mobile_money,successful,NIL,UK,C00057,2024-01-01,03:00,January,Early Hours
4,TXN000005,M0056,459.85,GBP,crypto,successful,NIL,UK,C00735,2024-01-01,04:00,January,Early Hours


In [53]:
merchants.head()

,merchant_id,merchant_name,merchant_category,country
0,M0001,Merchant_1,Gaming,Kenya
1,M0002,Merchant_2,Education,UK
2,M0003,Merchant_3,Saas,Kenya
3,M0004,Merchant_4,Travel,Nigeria
4,M0005,Merchant_5,Travel,UK


In [55]:
chargebacks.head()

,chargeback_id,transaction_id,merchant_id,chargeback_date,chargeback_amount,reason
0,CB00001,TXN000002,M0052,2024-02-01,404.26,fraud
1,CB00002,TXN000009,M0094,2024-02-02,819.55,fraud
2,CB00003,TXN000012,M0007,2024-02-03,366.68,product not received
3,CB00004,TXN000032,M0043,2024-02-04,399.92,duplicate transaction
4,CB00005,TXN000066,M0020,2024-02-05,77.49,product not received


In [49]:
# one big table data modelling for easy analysis
query = """
    CREATE TABLE transformed_datasets AS
    SELECT 
        t.transaction_id,
        t.merchant_id,
        t.transaction_amount,
        t.currency,
        t.payment_method,
        t.status,
        COALESCE(c.reason, t.failure_reason, 'NIL') AS failure_reason, -- Fill blank rows with chargeback reason
        m.merchant_name,
        m.merchant_category,
        m.country AS merchant_country,
        t.country AS transaction_country,
        t.customer_id,
        t.date AS transaction_date,
        c.chargeback_date,
        t.time,
        t.month,
        t.day_category,
        (JULIANDAY(c.chargeback_date) - JULIANDAY(t.date)) AS days_difference
    FROM transactions t
    LEFT JOIN merchants m ON t.merchant_id = m.merchant_id -- Keep all transactions
    LEFT JOIN chargebacks c ON t.transaction_id = c.transaction_id AND t.merchant_id = c.merchant_id;

"""
conn.execute(query)

conn.commit()

transformed = pd.read_sql("SELECT * FROM transformed_datasets", conn)
print(transformed.head())

  transaction_id merchant_id  transaction_amount currency payment_method  \
0      TXN000001       M0079              638.54      GBP    credit_card   
1      TXN000002       M0052              404.26      USD         crypto   
2      TXN000003       M0039               27.81      EUR    credit_card   
3      TXN000004       M0097              956.75      NGN   mobile_money   
4      TXN000005       M0056              459.85      GBP         crypto   

       status failure_reason merchant_name merchant_category merchant_country  \
0  successful            NIL   Merchant_79            Gaming          Nigeria   
1  chargeback          fraud   Merchant_52         Education          Germany   
2  successful            NIL   Merchant_39            Travel          Germany   
3  successful            NIL   Merchant_97           Finance               UK   
4  successful            NIL   Merchant_56            Gaming          Nigeria   

  transaction_country customer_id transaction_date charg

In [51]:
transformed.to_csv(r"C:\Users\New-user\Downloads\transformed_datasets.csv", index=False)

In [61]:
successful_df = transformed[transformed['status'] == 'successful']

# 1. Revenue Analysis
print("--- Revenue Analysis ---")
merchant_revenue = successful_df.groupby('merchant_id')['transaction_amount'].sum().sort_values(ascending=False)
print("Total revenue per merchant:")
print(merchant_revenue.head(10))

currency_revenue = successful_df.groupby('currency')['transaction_amount'].sum()
print("\nTotal revenue per currency:")
print(currency_revenue)

country_revenue = successful_df.groupby('transaction_country')['transaction_amount'].sum()
print("\nTotal revenue per country:")
print(country_revenue)

month_revenue = successful_df.groupby('month')['transaction_amount'].sum()
print("\nTotal revenue per month:")
print(month_revenue)

category_revenue = successful_df.groupby('merchant_category')['transaction_amount'].sum()
print("\nTotal revenue per merchant category:")
print(category_revenue)

payment_method_revenue = successful_df.groupby('payment_method')['transaction_amount'].sum()
print("\nTotal revenue per payment method:")
print(payment_method_revenue)


--- Revenue Analysis ---
Total revenue per merchant:
merchant_id
M0033    33669.44
M0054    29641.44
M0086    29282.37
M0026    29155.63
M0020    29000.67
M0060    27967.27
M0016    27833.98
M0099    27793.53
M0087    27060.76
M0051    26717.45
Name: transaction_amount, dtype: float64

Total revenue per currency:
currency
EUR    414214.53
GBP    443284.62
KES    424604.22
NGN    441363.27
USD    427867.02
Name: transaction_amount, dtype: float64

Total revenue per country:
transaction_country
Canada     379364.45
Germany    354607.08
Kenya      349190.21
Nigeria    353593.80
UK         356530.67
USA        358047.45
Name: transaction_amount, dtype: float64

Total revenue per month:
month
April       309332.78
February    297816.86
January     315549.56
July        259591.64
June        322717.81
March       328736.45
May         317588.56
Name: transaction_amount, dtype: float64

Total revenue per merchant category:
merchant_category
E-commerce    311766.85
Education     388271.20
Fina

In [63]:
# 2. Failure Rate Analysis
print("--- Failure Rate Analysis ---")
failed_df = transformed[transformed['status'] == 'failed']
print("Total revenue on failed transactions:", failed_df['transaction_amount'].sum())

failure_by_currency = failed_df['currency'].value_counts()
print("\nFailure frequency by currency:")
print(failure_by_currency)

failure_by_payment = failed_df['payment_method'].value_counts()
print("\nFailure frequency by payment method:")
print(failure_by_payment)

failure_by_country = failed_df['transaction_country'].value_counts()
print("\nFailure frequency by country:")
print(failure_by_country)

--- Failure Rate Analysis ---
Total revenue on failed transactions: 271017.18000000005

Failure frequency by currency:
currency
NGN    121
USD    111
GBP    105
EUR     96
KES     90
Name: count, dtype: int64

Failure frequency by payment method:
payment_method
mobile_money     150
bank_transfer    142
crypto           125
credit_card      106
Name: count, dtype: int64

Failure frequency by country:
transaction_country
Germany    101
USA         92
Nigeria     87
Canada      85
Kenya       80
UK          78
Name: count, dtype: int64


In [66]:
# 3. Chargeback & Fraud Analysis
print("--- Chargeback & Fraud Analysis ---")
chargeback_df = transformed[transformed['status'] == 'chargeback']
print("Total revenue lost to chargebacks:", chargeback_df['transaction_amount'].sum())

chargeback_by_reason = chargeback_df['failure_reason'].value_counts()
print("\nChargeback frequency by failure reason:")
print(chargeback_by_reason)

chargeback_by_payment = chargeback_df['payment_method'].value_counts()
print("\nChargeback frequency by payment method:")
print(chargeback_by_payment)


--- Chargeback & Fraud Analysis ---
Total revenue lost to chargebacks: 65844.70999999999

Chargeback frequency by failure reason:
failure_reason
fraud                    39
duplicate transaction    38
unauthorized             26
product not received     23
Name: count, dtype: int64

Chargeback frequency by payment method:
payment_method
crypto           35
credit_card      33
mobile_money     30
bank_transfer    28
Name: count, dtype: int64


In [68]:
# Fraud detection
fraudulent_chargebacks = chargeback_df.groupby('failure_reason')['transaction_amount'].sum()
print("\nChargeback revenue by failure reason:")
print(fraudulent_chargebacks)



Chargeback revenue by failure reason:
failure_reason
duplicate transaction    17941.00
fraud                    23245.87
product not received     11038.66
unauthorized             13619.18
Name: transaction_amount, dtype: float64


In [74]:
print("--- Additional Analysis ---")
print("Total transaction amount for the year:", transformed['transaction_amount'].sum())

currency_frequency = transformed['currency'].value_counts()
print("\nFrequency of each currency:")
print(currency_frequency)

payment_method_count = transformed['payment_method'].value_counts()
print("\nTotal transactions per payment method:")
print(payment_method_count)

status_payment_usage = transformed.groupby('status')['payment_method'].agg(pd.Series.mode)
print("\nMost used payment method per transaction status:")
print(status_payment_usage)


--- Additional Analysis ---
Total transaction amount for the year: 2546869.7800000003

Frequency of each currency:
currency
GBP    1029
NGN    1018
USD     996
EUR     982
KES     975
Name: count, dtype: int64

Total transactions per payment method:
payment_method
mobile_money     1265
bank_transfer    1254
credit_card      1245
crypto           1236
Name: count, dtype: int64

Most used payment method per transaction status:
status
chargeback          crypto
failed        mobile_money
refunded       credit_card
successful     credit_card
Name: payment_method, dtype: object


In [76]:
conn.close()